In [1]:
import tensorflow as tf
import os
import random
import sys
from PIL import Image
import numpy as np

In [2]:
number = ['0','1','2','3','4','5','6','7','8','9']
ALPHABET = ['A','B','C','D','E']
IMAGE_GEN_PATH = "./captcha/images/"

CHAR_SET=number + ALPHABET
print(CHAR_SET)

CHAR_MAP = {}
for index,value in enumerate(CHAR_SET):
    CHAR_MAP[value]=index
print(CHAR_MAP)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E']
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'A': 10, 'B': 11, 'C': 12, 'D': 13, 'E': 14}


In [3]:
tf.reset_default_graph()
#获取所有文件以及分类
def _get_filenames(dataset_dir):
    #数据目录
    photo_filenames = []
    #循环每个分类的文件夹
    for filename in os.listdir(dataset_dir):
        path = os.path.join(dataset_dir, filename)
        #把图片加入图片列表
        photo_filenames.append(path)
 
    return photo_filenames

def int64_feature(values):
    if not isinstance(values, (tuple, list)):
        values = [values]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=values))
 
def bytes_feature(values):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[values]))
 
def image_to_tfexample(image_data, image_format, labels):
    #Abstract base class for protocol messages.
    feature = {
        'image/encoded': bytes_feature(image_data),
        'image/format': bytes_feature(image_format),
    }
    for index,value in enumerate(labels):
        feature["image/label"+str(index)] = int64_feature(value),
    return tf.train.Example(features=tf.train.Features(feature=feature))

#把数据转为TFRecord格式
def _convert_dataset(split_name, filenames, dataset_dir):
    output_filename = _gettfrecord_filepath(dataset_dir,split_name)
    labels = [1,1,1,1]
    with tf.Session() as sess:
        with tf.python_io.TFRecordWriter(output_filename) as tfrecord_writer:
            for i,filename in enumerate(filenames):
                try:
                    sys.stdout.write('\r>> Converting image %d/%d' % (i+1, len(filenames)))
                    sys.stdout.flush()
                    #读取图片
                    image_data = Image.open(filename)
                    image_data = image_data.resize((224,224))
                    image_data = np.array(image_data.convert("L"))
                    image_data = image_data.tobytes()
                    labelstr = os.path.basename(filename)
                    labelstr = os.path.splitext(labelstr)[0]
                    for index,value in enumerate(labelstr):
                        labels[index] = CHAR_MAP[value]
                    #生成tfrecord文件
                    example = image_to_tfexample(image_data, b'jpg', labels)
                    tfrecord_writer.write(example.SerializeToString())
                except IOError as e:
                    print("Could not read:",filenames[i])
                    print("Error:",e)
                    print("Skip it\n")    
    sys.stdout.write('\n')
    sys.stdout.flush()
    
def _gettfrecord_filepath(dataset_dir,split_name):
    output_filename = os.path.join(dataset_dir,split_name+".tfrecord")
    return output_filename   

In [4]:
#生成tfrecord
TFRECORD_DIR = "./captcha/tfrecords"
_RANDOM_SEED = 0
_NUM_TEST = 1000

photo_filenames = _get_filenames(IMAGE_GEN_PATH)
random.seed(_RANDOM_SEED)
random.shuffle(photo_filenames)
training_filenames = photo_filenames[_NUM_TEST:]
testing_filenames = photo_filenames[:_NUM_TEST]

#数据转换
os.makedirs(TFRECORD_DIR,exist_ok=True)
_convert_dataset('train', training_filenames, TFRECORD_DIR)
_convert_dataset('test', testing_filenames, TFRECORD_DIR)
print("tfrecord 生成完毕。")

>> Converting image 44892/44892
>> Converting image 1000/1000
tfrecord 生成完毕。
